In [114]:
import requests
import time
import certifi
import pandas as pd

base_url = "https://api.scryfall.com/"

SET = 'dft'
PRICE = 2
search_query = f"s:{SET} usd<{PRICE} unique:prints -type:basic r>u -is:reprint"
search_url = f"{base_url}cards/search?q={search_query}"
headers = {
        "User-Agent": "scyrfall_tools/1.0 (jackson.travis.do@gmail.com)"  # Replace with your app name and email
    }

card_list = pd.DataFrame({'name':[],
                   'price':[], 
                   'fullart':[], 
                   'frame':[], 
                   'frame_effects': [], 
                   'border_color':[]
                   })

while search_url:
    response = requests.get(search_url, headers=headers, verify=False)
    if response.status_code == 200:
        data = response.json()
        
        # Print the names of the cards found in this page
        for card in data['data']:
            print(f"{card['name']} \t\t\t {card['prices']['usd']}")
            #for key in card.keys():
            #    if key not in df.columns:
            #        df[key] = None
            entry = {'name':card['name'], 'price':card['prices']['usd'], 'fullart':card['full_art'], 'frame':card['frame'], 'border_color':card['border_color']}
            if 'frame_effects' in card.keys():
                entry['frame_effects'] = card['frame_effects']
            card_list = pd.concat([card_list, pd.DataFrame([entry])], ignore_index=True)
        search_url = data.get('next_page', None)
        time.sleep(0.1)
    else:
        print("Error:", response.status_code, response.json().get("details"))





c:\src\scryfall-tools\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.scryfall.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Aatchik, Emerald Radian 			 0.23
Aatchik, Emerald Radian 			 0.58
Afterburner Expert 			 0.44
Afterburner Expert 			 0.46
Agonasaur Rex 			 1.07
Agonasaur Rex 			 1.08
Basri, Tomorrow's Champion 			 0.24
Basri, Tomorrow's Champion 			 0.28
Boommobile 			 0.17
Boommobile 			 0.20
Boommobile 			 None
Bulwark Ox 			 0.25
Bulwark Ox 			 0.21
Bulwark Ox 			 None
Burnout Bashtronaut 			 0.35
Burnout Bashtronaut 			 0.39
Captain Howler, Sea Scourge 			 0.38
Captain Howler, Sea Scourge 			 1.05
Caradora, Heart of Alacria 			 0.18
Caradora, Heart of Alacria 			 0.19
Coalstoke Gearhulk 			 0.91
Coalstoke Gearhulk 			 1.73
Count on Luck 			 0.36
Count on Luck 			 0.26
Cryptcaller Chariot 			 0.57
Cryptcaller Chariot 			 0.99
Cursecloth Wrappings 			 0.34
Cursecloth Wrappings 			 0.46
Daretti, Rocketeer Engineer 			 0.19
Daretti, Rocketeer Engineer 			 0.28
Debris Beetle 			 0.19
Debris Beetle 			 0.28
Demonic Junker 			 0.39
Demonic Junker 			 0.45
District Mascot 			 0.47
District Mascot 			 0.5

In [128]:
# Generate list for TCGplayer
import numpy as np

def get_tcgplayer_name(card):
    entry = card['name']
    if type(card['frame_effects']) == list:
        if 'showcase' in card['frame_effects']:
            entry = f"{entry} (Showcase)"
        if card['frame_effects'] == ['extendedart']:
            entry = f"{entry} (Extended)"
    if card['frame'] == '1997':
        entry = f"{entry} (Retro)"
    if card['border_color'] == 'borderless':
        entry = f"{entry} (Borderless)"
    return entry

buylist = pd.DataFrame()

for name in card_list['name'].unique():
    cards = card_list[card_list['name']==name]
    for index,card in cards.iterrows():
        entry = get_tcgplayer_name(card)
        buylist = pd.concat([buylist, pd.DataFrame([{'Quantity': 1, 'Name':entry, 'Price':card['price']}])], ignore_index=True)

    if len(cards) < 3:
        # Find cheapest printing
        to_add = cards[cards['name']==name].loc[cards['price'].idxmin()]
        buylist.loc[buylist['Name']==get_tcgplayer_name(to_add), 'Quantity'] += 1
        buylist = buylist.sort_values(by="Name")

buylist.to_csv('test.csv')

PermissionError: [Errno 13] Permission denied: 'test.csv'

In [101]:
buylist['Total Price'] = buylist['Quantity'].astype(int)*buylist['Price'].astype(float)
buylist['Total Price'].sum()

np.float64(98.31)

In [78]:
len(buylist[buylist['Name']=='Zealous Conscripts'])

1

In [13]:
card['foil']

True

In [105]:
buylist

,Quantity,Name,Price
0,2,Adaptive Omnitool,1.32
1,2,Adaptive Omnitool,1.12
2,2,Aetherflux Conduit,0.69
3,2,Aetherflux Conduit,0.60
4,2,Lost Monarch of Ifnir,0.96
5,2,Lost Monarch of Ifnir,0.82
6,2,"Nissa, Worldsoul Speaker",0.50
7,2,"Nissa, Worldsoul Speaker",0.53
8,2,On Wings of Gold,0.86
9,2,On Wings of Gold,0.93
